In [1]:
import pandas as pd
import requests
from io import StringIO

In [2]:

#On s'occupe d'abord de nettoyer les données sur le nombre de validation par jour à partir des données annuelles
url = "http://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false"
response = requests.get(url, verify=False)
df = pd.read_csv(StringIO(response.text), sep=';', header=None)
df.columns = ["ID", "Type", "Station", "Traffic", "Col4", "Col5", "Col6", "Col7", "Col8", "City", "Col10"]
subset_df = df[["Traffic", "Station"]]
subset_df["Traffic"], subset_df["Station"] = subset_df["Station"].copy(), subset_df["Traffic"].copy()
subset_df.columns = ["Station", "Traffic"]
subset_df = subset_df.iloc[1:].reset_index(drop=True)
# Créer un nouveau tableau avec la somme du trafic pour chaque station
sum_traffic_df = subset_df.groupby("Station")["Traffic"].sum().reset_index()
sum_traffic_df["Traffic"] = pd.to_numeric(sum_traffic_df["Traffic"], errors='coerce')
# Diviser le trafic par 365 pour chaque station pour avoir le trafic par jour
sum_traffic_df["Traffic_Per_Day"] = (sum_traffic_df["Traffic"] / 365).astype(int)


#On s'occupe ensuite de nettoyer les données sur la répartition de l'affluence sur une journée
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre (1).csv'
données_validation= pd.read_csv(url)
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['LIBELLE_ARRET', 'TRNC_HORR_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations', aggfunc='mean').reset_index()

#On fusionne les données des deux tableaux pour avoir le nombre de validation sur chaque plage horaire
merged_df = pd.merge(sum_traffic_df, donnees_organisees, left_on='Station', right_on='LIBELLE_ARRET')
columns_to_multiply = donnees_organisees.columns[1:]
resultat = merged_df.copy()
resultat[columns_to_multiply] = resultat[columns_to_multiply].multiply(resultat['Traffic_Per_Day'], axis=0)
resultat[columns_to_multiply] = resultat[columns_to_multiply].div(100)
resultat = resultat.iloc[:, 2:]
# Sélectionner les colonnes numériques
numeric_columns = resultat.select_dtypes(include='number').columns
resultat[numeric_columns] = resultat[numeric_columns].applymap(lambda x: int(x) if pd.notna(x) else x)
print(resultat)



/home/codespace/.python/current/lib/python3.10/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.ratp.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/tmp/ipykernel_17014/2693062597.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Traffic"], subset_df["Station"] = subset_df["Station"].copy(), subset_df["Traffic"].copy()


FileNotFoundError: [Errno 2] No such file or directory: 'validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre (1).csv'